In [12]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq
import json
import re

In [13]:
load_dotenv()

True

In [14]:
def get_llm():
    model = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.3,
    groq_api_key=os.getenv("GROQ_API_KEY"))
    return model

In [15]:
model=get_llm()

In [16]:
def chain():
    system_prompt="""You are an expert computer science educator and quiz designer.

Generate a set of exactly **10 multiple-choice questions** (MCQs) for students learning the topic: **{topic_name}**.

Follow these strict instructions:

1. Cover a wide range of important **subtopics** from the given topic.
2. The questions should be arranged by **difficulty level**:
   - 3 Easy questions (basic concepts and definitions)
   - 4 Medium questions (application-level, comparative questions)
   - 3 Advanced questions (analysis, multi-step reasoning, real-world problem-solving)
3. For each question, provide:
   - The question text
   - Exactly 4 answer options (labeled A, B, C, D)
   - The correct option label
   - A brief explanation of the correct answer

4. Format your output strictly in this JSON structure:
```json
[
  {{
    "level": "easy",
    "question": "Your question here",
    "options": {{
      "A": "Option A",
      "B": "Option B",
      "C": "Option C",
      "D": "Option D"
    }},
    "answer": "C",
    "explanation": "Your explanation here"
  }},
  ...
]
"""
    return system_prompt

In [17]:
def cleaner(llm_response):
    cleaned = re.sub(r"```json|```", "", llm_response).strip()
    answer=json.loads(cleaned)
    return answer

In [18]:
async def quiz_with_llm(llm,Topic):
    sys_prompt=chain()
    prompt=sys_prompt.format(topic_name=Topic)
    result=await llm.ainvoke(prompt)
    final_ans=cleaner(result.content)
    return final_ans

In [19]:
response=await quiz_with_llm(model,"Fundamentals of computers")

In [ ]:
print(response)

[{'level': 'easy', 'question': 'What does CPU stand for in the context of computer hardware?', 'options': {'A': 'Central Processing Unit', 'B': 'Central Power Supply', 'C': 'Computer Processing Unit', 'D': 'Central Processing Utility'}, 'answer': 'A', 'explanation': 'CPU stands for Central Processing Unit, which is the primary component of a computer responsible for executing instructions and handling data processing.'}, {'level': 'easy', 'question': 'Which of the following is an example of output device?', 'options': {'A': 'Keyboard', 'B': 'Mouse', 'C': 'Monitor', 'D': 'Microphone'}, 'answer': 'C', 'explanation': 'A monitor is an output device that displays visual information from the computer, whereas keyboard, mouse, and microphone are input devices.'}, {'level': 'easy', 'question': 'What is the primary function of the operating system?', 'options': {'A': 'To provide application software', 'B': 'To manage computer hardware and provide a platform for running applications', 'C': 'To c